# Prep

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import MinMaxScaler

## load deep learning framework

In [4]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout
from tensorflow.keras.utils import to_categorical 

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

## install and load audio processing tools

In [7]:
pip install librosa 

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 183 kB 186 kB/s eta 0:00:01     |███████████████████████▏        | 133 kB 186 kB/s eta 0:00:01
     |████████████████████████████████| 377 kB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 323 kB 250 kB/s eta 0:00:01
     |████████████████████████████████| 3.4 MB 289 kB/s eta 0:00:01
     |████████████████████████████████| 51 kB 203 kB/s eta 0:00:01
     |████████████████████████████████| 25.3 MB 186 kB/s eta 0:00:011
  Created wheel for librosa: filename=librosa-0.8.0-py3-none-any.whl size=201374 sha256=7e108a0e0ec20a466c1f08d51e78bb470756af12af23842ad11bb49f00d668d8
  Stored in directory: /home/yunwanx/.cache/pip/wheels/c6/08/26/9d7dec1c2dc863d8f1fdc1ac39883e2601dc6fa63fcd68183f
  Created wheel for audioread: filename=audioread-2.1.9-py3-none-any.whl size=23142 sha256=dcae8e6cbf6636daf11cd44bafb5638aed05daed79f0fbe04ca5c5e4f86f1caa
  Stored in directory: /home/yunwanx/.c

In [8]:
import os
import librosa
import librosa.display
import glob

In [9]:

feature = []
label = []
# 建立类别标签，不同类别对应不同的数字。
label_dict = {'aloe': 0, 'burger': 1, 'cabbage': 2,'candied_fruits':3, 'carrots': 4, 'chips':5,
                  'chocolate': 6, 'drinks': 7, 'fries': 8, 'grapes': 9, 'gummies': 10, 'ice-cream':11,
                  'jelly': 12, 'noodles': 13, 'pickles': 14, 'pizza': 15, 'ribs': 16, 'salmon':17,
                  'soup': 18, 'wings': 19}
label_dict_inv = {v:k for k,v in label_dict.items()}

In [10]:
from tqdm import tqdm
def extract_features(parent_dir, sub_dirs, max_file=10, file_ext="*.wav"):
    c = 0
    label, feature = [], []
    for sub_dir in sub_dirs:
        for fn in tqdm(glob.glob(os.path.join(parent_dir, sub_dir, file_ext))[:max_file]): # 遍历数据集的所有文件
            
           # segment_log_specgrams, segment_labels = [], []
            #sound_clip,sr = librosa.load(fn)
            #print(fn)
            label_name = fn.split('/')[-2]
            label.extend([label_dict[label_name]])
            X, sample_rate = librosa.load(fn,res_type='kaiser_fast')
            mels = np.mean(librosa.feature.melspectrogram(y=X,sr=sample_rate).T,axis=0) # 计算梅尔频谱(mel spectrogram),并把它作为特征
            feature.extend([mels])
            
    return [feature, label]

In [16]:

# 自己更改目录
parent_dir = '../data/train_sample/'
save_dir = "../output/"
folds = sub_dirs = np.array(['aloe','burger','cabbage','candied_fruits',
                             'carrots','chips','chocolate','drinks','fries',
                            'grapes','gummies','ice-cream','jelly','noodles','pickles',
                            'pizza','ribs','salmon','soup','wings'])

# 获取特征feature以及类别的label
temp = extract_features(parent_dir,sub_dirs,max_file=100)

100%|██████████| 35/35 [00:05<00:00,  6.06it/s]


In [17]:
temp = np.array(temp)
data = temp.transpose()

In [19]:

# 获取特征
X = np.vstack(data[:, 0])

# 获取标签
Y = np.array(data[:, 1])
print('X的特征尺寸是：',X.shape)
print('Y的特征尺寸是：',Y.shape)

X的特征尺寸是： (1000, 128)
Y的特征尺寸是： (1000,)


In [20]:

# 在Keras库中：to_categorical就是将类别向量转换为二进制（只有0和1）的矩阵类型表示
Y = to_categorical(Y)

In [21]:
'''最终数据'''
print(X.shape)
print(Y.shape)

(1000, 128)
(1000, 20)


In [22]:

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 1, stratify=Y)
print('训练集的大小',len(X_train))
print('测试集的大小',len(X_test))

训练集的大小 750
测试集的大小 250


In [23]:
X_train = X_train.reshape(-1, 16, 8, 1)
X_test = X_test.reshape(-1, 16, 8, 1)

# cnn

In [25]:

model = Sequential()
# 输入的大小
input_dim = (16, 8, 1)

model.add(Conv2D(64, (3, 3), padding = "same", activation = "tanh", input_shape = input_dim))# 卷积层
model.add(MaxPool2D(pool_size=(2, 2)))# 最大池化
model.add(Conv2D(128, (3, 3), padding = "same", activation = "tanh")) #卷积层
model.add(MaxPool2D(pool_size=(2, 2))) # 最大池化层
model.add(Dropout(0.1))
model.add(Flatten()) # 展开
model.add(Dense(1024, activation = "tanh"))
model.add(Dense(20, activation = "softmax")) # 输出层：20个units输出20个类的概率

# 编译模型，设置损失函数，优化方法以及评价标准
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [26]:

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 16, 8, 64)         640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 4, 64)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 4, 128)         73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 2, 128)         0         
_________________________________________________________________
dropout (Dropout)            (None, 4, 2, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              1

In [27]:
%%time
# 训练模型
model.fit(X_train, Y_train, epochs = 20, batch_size = 15, validation_data = (X_test, Y_test))

Epoch 1/20
50/50 [==============================] - 1s 28ms/step - loss: 2.8570 - accuracy: 0.1347 - val_loss: 2.6924 - val_accuracy: 0.1720
Epoch 2/20
50/50 [==============================] - 1s 23ms/step - loss: 2.5330 - accuracy: 0.2227 - val_loss: 2.5536 - val_accuracy: 0.2320
Epoch 3/20
50/50 [==============================] - 1s 24ms/step - loss: 2.3009 - accuracy: 0.2853 - val_loss: 2.4513 - val_accuracy: 0.2720
Epoch 4/20
50/50 [==============================] - 1s 26ms/step - loss: 2.0408 - accuracy: 0.3880 - val_loss: 2.3825 - val_accuracy: 0.2800
Epoch 5/20
50/50 [==============================] - 1s 23ms/step - loss: 1.8835 - accuracy: 0.4173 - val_loss: 2.4142 - val_accuracy: 0.3200
Epoch 6/20
50/50 [==============================] - 1s 25ms/step - loss: 1.7646 - accuracy: 0.4600 - val_loss: 2.3060 - val_accuracy: 0.3520
Epoch 7/20
50/50 [==============================] - 1s 24ms/step - loss: 1.5560 - accuracy: 0.5320 - val_loss: 2.5004 - val_accuracy: 0.3040
Epoch 8/20
50

# performance on test set

In [28]:

def extract_features(test_dir, file_ext="*.wav"):
    feature = []
    for fn in tqdm(glob.glob(os.path.join(test_dir, file_ext))[:]): # 遍历数据集的所有文件
        X, sample_rate = librosa.load(fn,res_type='kaiser_fast')
        mels = np.mean(librosa.feature.melspectrogram(y=X,sr=sample_rate).T,axis=0) # 计算梅尔频谱(mel spectrogram),并把它作为特征
        feature.extend([mels])
    return feature

In [29]:
X_test = extract_features('../data/test_a/')

100%|██████████| 2000/2000 [04:10<00:00,  8.00it/s]


In [31]:
X_test

array([[2.9598351e-03, 2.7406486e-03, 1.3819024e-03, ..., 4.6215209e-08,
        8.6533296e-09, 9.4653851e-10],
       [9.6117635e-04, 2.8952870e-03, 6.2826923e-03, ..., 3.5951460e-05,
        6.7042702e-06, 8.8306325e-07],
       [1.3216470e-02, 4.3380085e-02, 8.9629956e-02, ..., 2.9015726e-05,
        2.4174481e-06, 2.9420892e-07],
       ...,
       [3.8925678e-02, 2.7496809e-01, 2.9786795e-01, ..., 1.3696851e-04,
        3.6786754e-05, 3.8811245e-06],
       [2.0119630e-01, 3.0227860e-02, 1.2986433e-03, ..., 1.4483050e-05,
        3.6184322e-06, 3.9628182e-07],
       [2.4599414e+00, 1.5796322e+00, 1.3016220e+00, ..., 1.4945977e-05,
        2.7943854e-06, 2.9968697e-07]], dtype=float32)

In [30]:
X_test = np.vstack(X_test)
predictions = model.predict(X_test.reshape(-1, 16, 8, 1))